# Adaptive Boosting

### Motivation

在尋找問題答案的過程:

1. 先提出一個假設，讓在這個假設下錯誤的 False Positive &amp; True Negative 的樣本挑出放大
2. 針對被放大的樣本再進行下一個假設判斷，找出錯誤的在放大...

### Bootstrapping as Re-weighting Process

$$
\begin{align}
   & \mathcal{D} = \big\{ (x_1, y_1), (x_2, y_2), (x_3, y_3), (x_4, y_4) \big\} \\
\underbrace{\to}_{bootstrap} & \tilde{\mathcal{D}}_t = \big\{ (x_1, y_1), (x_1, y_1), (x_2, y_2), (x_4, y_4) \big\} 
\end{align}
$$

經過 bootstapping 隨機選出的資料 $ \tilde{D}_t $ 送進 $ E_{in} $；就和原來資料 D 送進加權過後的 weighted $ E_{in} $ 相同。

$$
E_{in}^u(h) = \frac{1}{4} \sum_{n=1}^4 u_n^{(t)} \cdot \big[ y_n \ne h(x_n) \big]_{boolean} \\
\\
(x_1,y_1), \ u_1 = 2 \\
(x_2,y_2), \ u_2 = 1 \\
(x_3,y_3), \ u_3 = 0 \\
(x_4,y_4), \ u_4 = 1
$$

如何把 $ u_n $ 放到 base algorithm 中?

### Weighted Base Algorithm

minimize (regularized):

$$
E_{in}^u(h) = \frac{1}{N} \sum_{n=1}^N u_n \cdot err \big( y_n, h(x_n) \big)
$$

#### 以 SVM 為例子:

by dual QP,

$
E_{in}^u \propto C \sum_{n=1}^N u_n \widehat{err}_{SVM} 
$

調整 upper bound 為 : $ 0 \le \alpha_n \le C u_n $

#### 以 Logistic Regression 為例子:

by SGD,

$
E_{in}^u \propto \sum_{n=1}^N u_n err_{CE} 
$

在 sample $ (x_n, y_n) $ 的時候，就依據 $ u_n $ 調整選中的機率。

### Re-weight for More Diverse Hypothesis

如何讓每一次 $ u_n $ sampling 採樣時候，都獲得較不同的樣本，讓兩輪的 $ g^t, g^{t+1} $ diverse ?

讓 $ g^{t} $ 結果，在 $ g^{t+1} $ 使用的資料上表現不好，在二元分類上，如同丟銅板只有 0.5 的正確率。

意即: # of 犯錯的 $ u_n^{t+1} $ = # of 正確的 $ u_n^{t+1} $

one possibility by re-scaling (multiplying) weights,

if total $ u_n^t $ of incorrect = 1126, total $ u_n^t $ of correct = 6211,

incorrect: $ u_n^{t+1} \leftarrow u_n^t \cdot 6211 $

correct: $ u_n^{t+1} \leftarrow u_n^t \cdot 1126 $

### Scaling Factor

'Optimal' re-weighting under weighted incorrect rate $ \epsilon_t $:  

$
\epsilon_t = \frac{ \sum_{n=1}^N u_n^{(t)} \big[ y_n \ne g_t(x_n) \big]_{boolean} }{ \sum_{n=1}^N u_n^{(t)} }
$

multiply incorrect $ \propto ( 1 - \epsilon_t ) $  
multiply correct $ \propto ( \epsilon_t ) $

define a scaling factor $ \blacklozenge_t $

$$
\blacklozenge_t = \sqrt{ \frac{1 - \epsilon_t}{\epsilon_t} }
$$

incorrect (t+1) &lt; &lt; &lt; incorrect (t) $ \times \blacklozenge_t $  
correct (t+1) &lt; &lt; &lt; correct (t) $ / \blacklozenge_t $

**equivalent** to optimal re-weighting

正常來說，學習的結果，錯誤率應該 $ \epsilon_t \le \frac{1}{2} $，因此 $ \blacklozenge_t \ge 1 $

$ \blacklozenge_t $ 會 scale up incorrect, scale down correct.

> Scaling-up incorrect examples leads to diverse hypotheses.

#### Adaptive Boosting Algorithm (AdaBoost)

Step 1 - obtain $ g_t $ by $ A( \mathcal{D}, u^{(t)} ) $, where A tries to minimize $ u^{(t)} $ - weighted 0/1 error.

Step 2 - update $ u^{(t)} \to u^{(t+1)} $ by $ \blacklozenge_t $, where $ \epsilon_t $ = weighted error (incorrect) rate of $ g_t $

NEXT: a special algorithm to aggregate linearly on the fly with theoretical guarentee.

Step 3 - compute $ \alpha_t = \ln( \blacklozenge_t ) $

Step 4 - return $ G(x) = sign \big( \sum_{t=1}^T \alpha_t g_t (x) \big) $

$ \epsilon_t = \frac{1}{2} \ \ \to \ \ \blacklozenge_t = 1 \ \ \to \ \ \alpha_t = 0 $, bad $ g_t $, ZERO weight.

$ \epsilon_t = 0 \ \ \to \ \ \blacklozenge_t = \infty \ \ \to \ \ \alpha_t = \infty $, super $ g_t $, superior weight.

### Decision Stump

A popular choice: decision stump

$$
h_{s,i,\theta} (x) = s \cdot sign \big( x_i - \theta \big)
$$

- positive &amp; negative rays on some feature: 3 parameters (feature i, threshold $ \theta $, direction s )
- physical meaning: Vertical / Horizontal lines in 2D
- efficient to optimize: $ O ( d \cdot N \log N ) $ time

### AdaBoost-Stump

AdaBoost + Decision Stump : efficient feature selection and aggregation. ( real-time face detection ).